# CM50267 Software Technologies for Data Science
### Lab 5 – Databases

In this lab you will develop an SQL database for the student assessment recording example we have used in the lectures. The lab is designed to be implemented using sqlite3 via python.

There is no auto-marker for this lab. You can have it marked in one of two ways.

Option 1: Demonstrate it to a tutor in the lab session. They will record that the code behaves correctly and you will receive the mark. You *must* also upload it to moodle by the lab deadline. No marks will be given if it is not uploaded, even if the tutor has checked the code. This is the fastest way to get feedback, and ensure that you will get the marks.

Option 2: Upload the code to moodle by the lab deadline. It will be marked online using an additional dataset. This will be a slower process and you run the risk that even though your code produced the output below, there is a bug that will be demonstrated by the use of a different dataset.


In [3]:
# This code wraps up the database access in a single function.
import sqlite3

# access_database requires the name of a sqlite3 database file and the query.
# It does not return the result of the query.
def access_database(dbfile, query):
    connect = sqlite3.connect(dbfile)
    cursor = connect.cursor()
    cursor.execute(query)
    connect.commit()
    connect.close()

# access_database requires the name of an sqlite3 database file and the query.
# It returns the result of the query
def access_database_with_result(dbfile, query):
    connect = sqlite3.connect(dbfile)
    cursor = connect.cursor()
    rows = cursor.execute(query).fetchall()
    connect.commit()
    connect.close()
    return rows

In [5]:
a=[(1,)]
a[0][0]

1

## Initial Setup

The first step is to create the tables needed to store the data. These will be pre-defined to ensure consistency.

### units
This table holds the basic detail of a unit.

unitid A unique integer identifier used to reference a unit.

name The sring name of the unit.

level The educational level of the unit.

semester The semester in which the unit occurs. 0 indicated whole year.

### students
This table holds the basic detail of a student.

studentid A unique integer identifier used to reference a student.

name A string name of the student.

### enrolled
This table indicates which students are enrolled on a unit and in which year. Note that it used the id values of the student and unit to create the relatioship.

studentid The id of a student.

unitid The id of a unit.

year An integer indicate the year the student was enrolled in the unit.

### assessments
An assessment if an assignement create by the lecturer for a unit, in a given year. The weightings for a unit in a given year are expected to sum to 100%

assessmentid A unique integer identifier used to reference an assignment.

unitd Which unit this assessment belongs to.

year The integer year that this assignment relates to.

name A string name for the assessment.

mark The integer maximum mark available for the assignment.

weighting The integer weighting out of 100 for this assignment.

deadline The initial deadline set for this assignment.

### assignments
An assignment is the record of the assessment for an individual student.

assignmentid A unique integer identifier used to reference an assignment instance.
studentid The student this instance relates to.

assessmentid The assessment this instance relates to.

deadline The deadline for this student.

submitted The date on which this assessment was submitted.

mark The mark allocated to this piece of work.

marked A flag indicating if this piece of work has been marked. 0:no, 1: yes.

In [2]:
# Set up the tables

def setup_assessment_tables(dbfile):
    # Get rid of any existing data
    access_database(dbfile,"CREATE TABLE IF NOT EXISTS users (userid INTEGER PRIMARY KEY, username TEXT NOT NULL, password TEXT NOT NULL)")
    access_database(dbfile,"CREATE TABLE IF NOT EXISTS session (sessionid INTEGER PRIMARY KEY, userid INTEGER, magic TEXT NOT NULL, start INTEGER, end INTEGER)")
    access_database(dbfile,"CREATE TABLE IF NOT EXISTS traffic (recordid INTEGER PRIMARY KEY, sessionid INTEGER, time INTEGER, type INTEGER, occupancy INTEGER, location TEXT NOT NULL, mode INTEGER)")
    access_database(dbfile,"INSERT INTO users VALUES(1,'test1','password1')")
    access_database(dbfile,"INSERT INTO users VALUES(2,'test2','password2')")
    access_database(dbfile,"INSERT INTO users VALUES(3,'test3','password3')")
    access_database(dbfile,"INSERT INTO users VALUES(4,'test4','password4')")
    access_database(dbfile,"INSERT INTO users VALUES(5,'test5','password5')")
    access_database(dbfile,"INSERT INTO users VALUES(6,'test6','password6')")
    access_database(dbfile,"INSERT INTO users VALUES(7,'test7','password7')")
    access_database(dbfile,"INSERT INTO users VALUES(8,'test8','password8')")
    access_database(dbfile,"INSERT INTO users VALUES(9,'test9','password9')")
    access_database(dbfile,"INSERT INTO users VALUES(10,'test10','password10')")

In [ ]:
setup_ass

In [3]:
# Task 1 (1 Mark): Provide a function that indicates the units a student is taking in a given year.
# It should return a list of the units being taken in icreasing order of unitid. e.g. [101,104,105]

def student_units(database, studentid, year):
    row = access_database_with_result(database,"SELECT unitid from enrolled WHERE year= {} and studentid= {} order by unitid;".format(year,studentid))
    l = []
    for u in row:
        l.append(u[0])
    return l


In [4]:
# Task 2 (1 Mark): Provide a function that indicates the students that are enrolled on a unit.
# It should return a list of tupples of studentid and name ordered by studentid. e.g. [(110,'Zipppy'),(111,'Bungle')]

def unit_students(database, unitid, year):
    row = access_database_with_result(database,"SELECT students.studentid ,name from students INNER JOIN enrolled on students.studentid=enrolled.studentid where enrolled.unitid={} and enrolled.year={};".format(unitid,year))
    return row
        

In [5]:
# Task 3 (2 Marks): Provide a function that indicates how many students are taking each unit in a given year.
# It should return a list of tupples of unitid, unitname and count ordered by sunitid. e.g. [(1010,'Machine Learning',50),(1020,'Dissertaton',37)]

def unit_numbers(database, year):
    row = access_database_with_result(database,"SELECT units.unitid,units.name,count(enrolled.unitid) from units INNER JOIN enrolled on enrolled.unitid=units.unitid WHERE enrolled.year= {} group by enrolled.unitid;".format(year))
    return row


In [6]:
# Task 4 (2 Marks): Provide a function that uses the enrollments and assessements tables to fully populate the assignments table.
# All asssignemntsfor each student in a given year should be created. Only the assignements a student should be
# undertaking should be created.

def create_assignments(database, year):
    access_database(database,"INSERT INTO assignments (studentid, assessmentid, deadline, marked) SELECT enrolled.studentid, assessments.assessmentid, assessments.deadline, 0  FROM enrolled INNER JOIN assessments ON enrolled.unitid = assessments.unitid AND enrolled.year = assessments.year WHERE enrolled.year ={};".format(year))


In [7]:
# Task 5 (2 Mark): Update the mark of an assignment, given the studentid, assessmentid and mark.
# It should update the marked flag and the mark.

def mark_assignment(database, studentid, assessmentid, mark):
    access_database(database,"UPDATE assignments SET mark = {}, marked=1 WHERE studentid = {} AND assessmentid = {}".format(mark, studentid, assessmentid))



In [8]:
# Task 6 (2 Marks): Compute the overall mark for all students taking a specified unit in a given year.

def unit_marks(database, unitid, year):
    rows = access_database_with_result(database,"SELECT assignments.studentid , sum(assignments.mark/assessments.mark*assessments.weighting) from assignments INNER JOIN assessments on assignments.assessmentid = assessments.assessmentid WHERE assessments.unitid={} and assessments.year={} GROUP by assignments.studentid;".format(unitid,year))
    return rows


In [9]:
# Task 7 (2 Marks): Compute the overall marks for each unit taken by a given student across all years.

def student_marks(database, studentid):
    rows=access_database_with_result(database,"SELECT assessments.unitid ,year, round(sum(assignments.mark/assessments.mark*assessments.weighting),2) from assignments INNER JOIN assessments on assignments.assessmentid = assessments.assessmentid WHERE assignments.studentid={} GROUP by assessments.unitid ;".format(studentid));
    return rows


In [10]:
# This simple test exercises each of the function you need to complete.
def run_test():
    setup_assessment_tables("example.db")

    result = student_units("example.db",1001,2020)
    print(result)

    result = unit_students("example.db",101,2020)
    print(result)

    result = unit_numbers("example.db",2020)
    print(result)

    create_assignments("example.db",2020)
    print(access_database_with_result("example.db","SELECT * FROM assignments;"))

    mark_assignment("example.db",1001, 1,57);
    mark_assignment("example.db",1001, 2,11);
    mark_assignment("example.db",1001, 3,45);
    mark_assignment("example.db",1002, 3,40);
    print(access_database_with_result("example.db","SELECT * FROM assignments;"))

    print(unit_marks("example.db",101, 2020))

    print(student_marks("example.db",1001))
    


In [11]:
setup_assessment_tables("example.db")
result = student_units("example.db",1001,2020)
print(result)

result = unit_students("example.db",101,2020)
print(result)

result = unit_numbers("example.db",2020)
print(result)

create_assignments("example.db",2020)
print(access_database_with_result("example.db","SELECT * FROM assignments;"))

mark_assignment("example.db",1001, 1,57);
mark_assignment("example.db",1001, 2,11);
mark_assignment("example.db",1001, 3,45);
mark_assignment("example.db",1002, 3,40);
print(access_database_with_result("example.db","SELECT * FROM assignments;"))

print(unit_marks("example.db",101, 2020))

print(student_marks("example.db",1001))

[100, 101]
[(1001, 'Rod'), (1002, 'Jane')]
[(100, 'CM60100', 1), (101, 'CM60101', 2)]
[(1, 1001, 1, '2021-1-25 20:00', None, None, 0), (2, 1001, 2, '2020-12-25 20:00', None, None, 0), (3, 1001, 3, '2020-12-15 20:00', None, None, 0), (4, 1002, 3, '2020-12-15 20:00', None, None, 0)]
[(1, 1001, 1, '2021-1-25 20:00', None, 57.0, 1), (2, 1001, 2, '2020-12-25 20:00', None, 11.0, 1), (3, 1001, 3, '2020-12-15 20:00', None, 45.0, 1), (4, 1002, 3, '2020-12-15 20:00', None, 40.0, 1)]
[(1001, 90.0), (1002, 80.0)]
[(100, 2020, 45.5), (101, 2020, 90.0)]


In [12]:
# This cell runs the tests in the cell above.
run_test()

[100, 101]
[(1001, 'Rod'), (1002, 'Jane')]
[(100, 'CM60100', 1), (101, 'CM60101', 2)]
[(1, 1001, 1, '2021-1-25 20:00', None, None, 0), (2, 1001, 2, '2020-12-25 20:00', None, None, 0), (3, 1001, 3, '2020-12-15 20:00', None, None, 0), (4, 1002, 3, '2020-12-15 20:00', None, None, 0)]
[(1, 1001, 1, '2021-1-25 20:00', None, 57.0, 1), (2, 1001, 2, '2020-12-25 20:00', None, 11.0, 1), (3, 1001, 3, '2020-12-15 20:00', None, 45.0, 1), (4, 1002, 3, '2020-12-15 20:00', None, 40.0, 1)]
[(1001, 90.0), (1002, 80.0)]
[(100, 2020, 45.5), (101, 2020, 90.0)]


## This is the output that run_test() should generate for the example shown.

[100, 101]

[(1001, 'Rod'), (1002, 'Jane')]

[(100, 'CM60100', 1), (101, 'CM60101', 2)]

[(1, 1001, 1, '2021-1-25 20:00', None, None, 0), (2, 1001, 2, '2020-12-25 20:00', None, None, 0), (3, 1001, 3, '2020-12-15 20:00', None, None, 0), (4, 1002, 3, '2020-12-15 20:00', None, None, 0)]

[(1, 1001, 1, '2021-1-25 20:00', None, 57, 1), (2, 1001, 2, '2020-12-25 20:00', None, 11, 1), (3, 1001, 3, '2020-12-15 20:00', None, 45, 1), (4, 1002, 3, '2020-12-15 20:00', None, 40, 1)]

[(1001, 90.0), (1002, 80.0)]

[(100, 2020, 45.5), (101, 2020, 90.0)]
